# Two‐Way ANOVA and Local Marker Analysis

This notebook:
1. Performs a two‐way ANOVA on per‐frame angle‐prediction errors across four global pipelines:
   - Plain ArUco (no Kalman)
   - ArUco + Kalman
   - QC (no Kalman)
   - QC + Kalman  
   with factors:
   - **Algorithm**: Plain vs QC
   - **Kalman**: No vs Yes  
2. Explains the ANOVA results.  
3. Performs a one‐way ANOVA across markers to identify which marker behaves best (local analysis).  
4. Provides interpretation of the outputs.  


## 1. Imports

Import necessary Python libraries.


In [1]:
import pandas as pd
import numpy as np
import cv2
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f_oneway
%matplotlib inline


## 2. Load Ground Truth & Extract Euler Angles

Load the ground truth CSV (`6DoF_annotated_poses.csv`) and convert its Rodrigues vectors (`rglob_x, rglob_y, rglob_z`) to Euler angles (roll, pitch, yaw) in degrees.  

The ZYX‐convention formulas are:
$$
\mathbf{R} = \mathrm{Rodrigues}(r),
\quad
\text{roll} = \atan2(R_{2,1}, R_{2,2}),
\quad
\text{pitch} = \atan2(-R_{2,0}, \sqrt{R_{0,0}^2 + R_{1,0}^2}),
\quad
\text{yaw} = \atan2(R_{1,0}, R_{0,0}).
$$

If $\sqrt{R_{0,0}^2 + R_{1,0}^2} < 10^{-6}$ (near gimbal lock), set $\text{yaw}=0$ and compute roll/pitch via an alternate formula.


In [2]:
def load_gt(gt_path):
    df = pd.read_csv(gt_path)
    required = {'frame', 'rglob_x', 'rglob_y', 'rglob_z', 'input_angle'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"GT missing columns: {missing}")
    return df

def extract_euler(df):
    df_u = df.drop_duplicates('frame').reset_index(drop=True)
    rvecs = df_u[['rglob_x', 'rglob_y', 'rglob_z']].to_numpy(dtype=np.float32)
    rolls, pitches, yaws = [], [], []
    for r in rvecs:
        R, _ = cv2.Rodrigues(r)
        sy = np.sqrt(R[0,0]**2 + R[1,0]**2)
        if sy > 1e-6:
            roll  = np.degrees(np.arctan2(R[2,1], R[2,2]))
            pitch = np.degrees(np.arctan2(-R[2,0], sy))
            yaw   = np.degrees(np.arctan2(R[1,0], R[0,0]))
        else:
            roll  = np.degrees(np.arctan2(-R[1,2], R[1,1]))
            pitch = np.degrees(np.arctan2(-R[2,0], sy))
            yaw   = 0.0
        rolls.append(roll)
        pitches.append(pitch)
        yaws.append(yaw)
    return pd.DataFrame({'frame': df_u['frame'], 'input_angle': df_u['input_angle'],
                         'roll_gt': rolls, 'pitch_gt': pitches, 'yaw_gt': yaws})

# Load GT
gt_path = '../CSVs/6DoF_annotated_poses.csv'
df_gt = load_gt(gt_path)
df_angles = extract_euler(df_gt)
df_angles.head()


,frame,input_angle,roll_gt,pitch_gt,yaw_gt
0,0,120.0,-116.520302,-10.361492,-34.930893
1,1,120.0,-116.304588,-10.342566,-34.908001
2,2,120.0,-116.316025,-10.320197,-34.871689
3,3,119.0,-116.462685,-10.310642,-34.966785
4,4,119.0,-116.315163,-10.241025,-34.883057


## 3. Global Pipelines: Compute Errors & Two‐Way ANOVA

We have four global pipelines:
- **Plain_NoKalman**: `plain_aruco_global.csv` (Plain ArUco, no Kalman)  
- **Plain_Kalman**: `aruco_kalman_global.csv` (Plain ArUco, with Kalman)  
- **QC_NoKalman**: `qc_global.csv` (Quality‐Controlled, no Kalman)  
- **QC_Kalman**: `qc_kalman_global.csv` (Quality‐Controlled, with Kalman)  

**Pipeline regression equation:**  
$$
\text{input\_angle} \approx a\,\text{roll}_{\text{pred}} + b\,\text{pitch}_{\text{pred}} + c\,\text{yaw}_{\text{pred}} + d.
$$

For each pipeline:
1. Merge predicted `(roll, pitch, yaw)` with `df_angles` on `frame`.  
2. Fit the above linear model.  
3. Compute `error = \text{pred\_angle} - \text{input\_angle}`.  
4. Tag each row with:
   - `algorithm`: “Plain” or “QC”  
   - `kalman`: “No” or “Yes”  
5. Combine all pipelines into a single DataFrame for ANOVA.

**Two‐Way ANOVA model:**
$$
error_{ijk} = \mu + \alpha_i + \beta_j + (\alpha\beta)_{ij} + \varepsilon_{ijk},
$$
where:
- $i \in \{\text{Plain}, \text{QC}\}$ (Algorithm)  
- $j \in \{\text{No}, \text{Yes}\}$ (Kalman)  
- $(\alpha\beta)_{ij}$ is the interaction term  
- $\varepsilon_{ijk}$ are residuals  


In [3]:
pipelines = {
    'Plain_NoKalman': ('plain_aruco_global.csv', 'Plain', 'No'),
    'Plain_Kalman':   ('aruco_kalman_global.csv', 'Plain', 'Yes'),
    'QC_NoKalman':    ('qc_global.csv', 'QC', 'No'),
    'QC_Kalman':      ('qc_kalman_global.csv', 'QC', 'Yes')
}

combined_rows = []
for name, (path, algo, kal) in pipelines.items():
    df_pred = pd.read_csv(path)
    # Merge on 'frame'
    df_merge = pd.merge(df_angles, df_pred[['frame','roll','pitch','yaw']], on='frame')
    df_merge = df_merge.rename(columns={'roll':'roll_pred', 'pitch':'pitch_pred', 'yaw':'yaw_pred'})
    # Fit regression
    X = df_merge[['roll_pred','pitch_pred','yaw_pred']].to_numpy()
    y = df_merge['input_angle'].to_numpy()
    model = LinearRegression().fit(X, y)
    df_merge['pred_angle'] = model.predict(X)
    df_merge['error'] = df_merge['pred_angle'] - df_merge['input_angle']
    # Tag categories
    df_merge['algorithm'] = algo
    df_merge['kalman'] = kal
    combined_rows.append(df_merge[['frame','error','algorithm','kalman']])

df_combined = pd.concat(combined_rows, ignore_index=True)
df_combined.head()


,frame,error,algorithm,kalman
0,1,-1.342285,Plain,No
1,2,-1.444160,Plain,No
2,3,-0.471801,Plain,No
3,4,-0.323887,Plain,No
4,5,-0.451797,Plain,No


### 3.1 Two‐Way ANOVA

We fit the model:
$$
error \sim C(algorithm) + C(kalman) + C(algorithm):C(kalman).
$$


In [4]:
model = ols('error ~ C(algorithm) + C(kalman) + C(algorithm):C(kalman)', data=df_combined).fit()
anova_results = sm.stats.anova_lm(model, typ=2)
print("### Two‐Way ANOVA Table (Global Pipelines)")
anova_results


### Two‐Way ANOVA Table (Global Pipelines)


,sum_sq,df,F,PR(>F)
C(algorithm),2.170955e-25,1.0,5.183436e-26,1.0
C(kalman),4.400535e-29,1.0,1.050684e-29,1.0
C(algorithm):C(kalman),2.838738e-29,1.0,6.777854e-30,1.0
Residual,4.372539e+03,1044.0,NaN,NaN


### 3.2 Interpretation of Two‐Way ANOVA Results

Below is an example of the ANOVA output you might see:

| Source                         | sum_sq    | df | F        | PR(>F) |
| ------------------------------ | --------- | -- | -------- | ------ |
| C(algorithm)                   | 2.17e-25  | 1  | 5.18e-26 | 1.0000 |
| C(kalman)                      | 4.60e-29  | 1  | 1.05e-29 | 1.0000 |
| C(algorithm):C(kalman)         | 2.84e-29  | 1  | 6.78e-30 | 1.0000 |
| Residual                       | 4.37e+03  | 1044 | —      | —     |

- **sum_sq** close to zero for each factor means that switching between Plain vs QC or No vs Yes‐Kalman explains virtually no variance in the error.  
- **F**‐statistics are essentially zero, and **PR(>F)** (p‐values) are 1.000, signifying no significant effect of either factor or their interaction.  
- **Residual** sum of squares (~4372) is the total unexplained variance, dominating the table.  

**Conclusion:**  
> All four pipelines (Plain_NoKalman, Plain_Kalman, QC_NoKalman, QC_Kalman) have statistically identical error distributions. Neither “Algorithm” nor “Kalman” (nor their interaction) affects mean error in a significant way.


## 4. Local Marker Analysis: One‐Way ANOVA Across Markers

We ask: For a single pipeline (Plain ArUco without Kalman), do different markers produce significantly different mean errors?

**Steps:**
1. Load `plain_aruco_local.csv`, which contains `(frame, marker_id, roll, pitch, yaw)`.  
2. Merge with GT local Euler (`rloc_x, rloc_y, rloc_z`) and `input_angle`.  
3. Fit `error = pred_angle - input_angle` for each `(frame, marker_id)`.  
4. Perform a one‐way ANOVA on `error` grouped by `marker_id`.


In [6]:
# 4.1 Load and Convert GT local Rodrigues to Euler per marker
df_gt_local = df_gt[['frame','marker_id','rloc_x','rloc_y','rloc_z','input_angle']].drop_duplicates(['frame','marker_id']).reset_index(drop=True)

def rvecs_to_euler(rvec):
    R, _ = cv2.Rodrigues(rvec.astype(np.float32))
    sy = np.sqrt(R[0,0]**2 + R[1,0]**2)
    if sy > 1e-6:
        roll  = np.degrees(np.arctan2(R[2,1], R[2,2]))
        pitch = np.degrees(np.arctan2(-R[2,0], sy))
        yaw   = np.degrees(np.arctan2(R[1,0], R[0,0]))
    else:
        roll  = np.degrees(np.arctan2(-R[1,2], R[1,1]))
        pitch = np.degrees(np.arctan2(-R[2,0], sy))
        yaw   = 0.0
    return roll, pitch, yaw

rolling, pitching, yawing = [], [], []
for vec in df_gt_local[['rloc_x','rloc_y','rloc_z']].to_numpy():
    r, p, y_ = rvecs_to_euler(vec)
    rolling.append(r); pitching.append(p); yawing.append(y_)
df_gt_local['roll_gt'] = rolling
df_gt_local['pitch_gt'] = pitching
df_gt_local['yaw_gt'] = yawing


### 4.2 Compute Errors for Each Marker (Plain ArUco, No Kalman)

Load `plain_aruco_local.csv` and compute the error for each `(frame, marker_id)`.


In [8]:
# Load plain ArUco local predictions
df_local_pred = pd.read_csv('../plain_aruco_local.csv')

# Merge GT local Euler with predicted local Euler
df_pred_local_euler = df_local_pred[['frame','marker_id','roll','pitch','yaw']].rename(
    columns={'roll':'roll_pred','pitch':'pitch_pred','yaw':'yaw_pred'}
)
df_merge_local = pd.merge(df_gt_local, df_pred_local_euler, on=['frame','marker_id'])

# Fit regression per marker and collect errors
marker_errors = {}
marker_stats = []
for mid in sorted(df_merge_local['marker_id'].unique()):
    df_m = df_merge_local[df_merge_local['marker_id'] == mid].copy()
    X_m = df_m[['roll_pred','pitch_pred','yaw_pred']].to_numpy()
    y_m = df_m['input_angle'].to_numpy()
    model_m = LinearRegression().fit(X_m, y_m)
    df_m['pred_angle'] = model_m.predict(X_m)
    df_m['error'] = df_m['pred_angle'] - df_m['input_angle']
    errs_m = df_m['error'].values
    marker_errors[mid] = errs_m
    marker_stats.append({
        'marker_id': mid,
        'n': len(errs_m),
        'mean_error': errs_m.mean(),
        'std_error': errs_m.std(ddof=1)
    })

df_marker_stats = pd.DataFrame(marker_stats)
print("### Marker‐Wise Error Statistics (Plain ArUco, No Kalman)")
df_marker_stats


### Marker‐Wise Error Statistics (Plain ArUco, No Kalman)


,marker_id,n,mean_error,std_error
0,2,262,-2.143561e-13,14.472648
1,4,262,-8.678385e-16,2.703391
2,8,155,9.168293e-16,4.928306
3,9,260,1.377360e-14,2.561386
4,10,27,5.526444e-15,6.056840


### 4.3 One‐Way ANOVA Across Markers

Test whether mean error differs among markers (IDs).


In [9]:
# Perform one-way ANOVA if we have multiple markers
errors_list = [marker_errors[mid] for mid in sorted(marker_errors)]
if len(errors_list) > 1:
    f_stat_m, p_val_m = f_oneway(*errors_list)
    print(f"ANOVA (Across Markers): F = {f_stat_m:.4f}, p = {p_val_m:.4e}")
else:
    print("Not enough markers to perform ANOVA.")


ANOVA (Across Markers): F = 0.0000, p = 1.0000e+00


## 5. Conclusion

- **Global Two‐Way ANOVA**: Evaluates main effects of Algorithm (Plain vs QC) and Kalman (No vs Yes), and their interaction, on angle‐prediction error.  
  - Because the p‐values are all 1.0, none of the factors (algorithm, kalman, or their interaction) significantly affect the mean error. All four pipelines have virtually identical error distributions.  
- **Local One‐Way ANOVA**: Tests whether mean error differs across marker IDs (for Plain ArUco without Kalman).  
  - All marker means are effectively zero, so the ANOVA p‐value is ~1.0. No marker has a systematically different mean error.  
- **Note on variances**: Although means are identical, the standard deviations vary (e.g., marker 9 is the most precise). If you wish to compare precision, consider comparing standard deviations or using Levene’s test rather than ANOVA on means.  
